In [5]:
# read json file def
def read_json_file(path):
    import json

    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)


In [4]:
path = "ALQAC_2025_data/alqac25_train.json"
json_data = read_json_file(path)


In [3]:
json_data[2]

{'question_id': 'train_alqac25_3',
 'question_type': 'Đúng/Sai',
 'text': 'Trường hợp thông tin trong Sổ hộ khẩu còn hiệu lực có thông tin khác với thông tin trong Cơ sở dữ liệu về cư trú thì sử dụng thông tin trong Cơ sở dữ liệu về cư trú, đúng hay sai?',
 'relevant_articles': [{'law_id': 'Luật Cư trú', 'article_id': '38'}],
 'answer': 'Đúng'}

In [6]:
base_line = read_json_file("evaluation_task2/result_task_2_baseline.json")
llama = read_json_file("evaluation_task2/result_task_2_Llama-3.1-8B-Instruct.json")
gemma = read_json_file("evaluation_task2/result_task_2_gemma-9b-it.json")
deepseek = read_json_file(
    "evaluation_task2/result_task_2_DeepSeek-R1-Distill-Llama-8B.json"
)
base_line_tot = read_json_file("evaluation_task2/result_task_2_baseline_ToT.json")

In [7]:
base_line_results = base_line["results"]
llama_results = llama["results"]
gemma_results = gemma["results"]
deepseek_results = deepseek["results"]
base_line_tot_results = base_line_tot["results"]

In [20]:
from collections import Counter


def combine_results(result_list, best_model_index, json_data):
    final_results = []

    # Check if all lists have the same length
    lengths = [len(results) for results in result_list]
    if len(set(lengths)) != 1:
        print("Error: Result lists have different lengths.")
        return []

    for i in range(lengths[0]):
        question_type = json_data[i]["question_type"]
        final_predict = None

        if question_type == "Tự luận":
            # For essay questions, take prediction from last model
            final_predict = result_list[best_model_index][i]["predict"]
        else:
            # For other types, get majority vote from all models
            predicts = [results[i]["predict"] for results in result_list]
            counts = Counter(predicts)
            final_predict = counts.most_common(1)[0][0]

        result_item = {
            "question_id": json_data[i]["question_id"],
            "answer": final_predict,
            "question_type": question_type,
        }
        final_results.append(result_item)

    return final_results


# Usage:
final_results = combine_results(
    [base_line_results, llama_results, gemma_results, base_line_tot_results],
    2,
    json_data,
)

In [21]:
def acc_eval_from_list(prediction_list, gold_list):
    """
    Evaluate accuracy from prediction and gold answer lists.
    Each element in both lists is a dict with keys: 'question_id' and 'answer'.
    Returns accuracy float.
    """
    gold_dict = {item["question_id"]: item["answer"] for item in gold_list}
    pred_dict = {item["question_id"]: item["answer"] for item in prediction_list}
    count_true = 0
    total = len(gold_dict)
    for qid, gold_ans in gold_dict.items():
        pred_ans = pred_dict.get(qid)
        if pred_ans is not None and pred_ans.lower() == gold_ans.lower():
            count_true += 1
    acc = count_true / total if total else 0
    print("Acc = {}".format(acc))
    return acc


In [22]:
acc = acc_eval_from_list(final_results, json_data)
eval_metrics = {
    "accuracy": acc,
    "total_questions": len(json_data),
    "evaluated_questions": len(final_results),
}

Acc = 0.7722908093278463
